In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# # Extract features using MetaMap
# from pymetamap import MetaMap
# mm = MetaMap.get_instance('/Users/rabinakhatiwada/Documents/pymetamap-master/public_mm/bin/metamap16')

### Sample Implementation

In [ ]:
# sents = ['Sleep Apnea Obstructive']
# concepts,error = mm.extract_concepts(sents,[1,2])
# for concept in concepts:
#     types = concept.semtypes.strip("[]").split(",") # Returns an iterable list of all semtypes
#     for type in types:
#         print(type)

In [ ]:
# text = "Mrs. Jones came in today complaining of a lot of chest pain. She denies shortness of breath and fevers."
# concepts, error = mm.extract_concepts([text])
# for concept in concepts:
#     print(concept)

### Loading dataset

In [ ]:
# READ IN DATASET
data = pd.read_csv('data.csv')
data = data.dropna(subset=['text', 'label']) # Remove all rows with missing values
data['text'] = data['text'].str.replace(r'[^\w\s]+', '', regex=True) # Remove all punctuation
data['text'] = data['text'].str.encode('ascii', 'ignore').str.decode('ascii') # Remove all non-ASCII chars
data['index'] = data.index

In [ ]:
# Declare entity types as a list
entity_types = [
    'aapp', 'acab', 'acty', 'aggp', 'amas', 'amph', 'anab', 'anim', 'anst', 'antb',
    'arch', 'bacs', 'bact', 'bdsu', 'bdsy', 'bhvr', 'biof', 'bird', 'blor', 'bmod',
    'bodm', 'bpoc', 'bsoj', 'celc', 'celf', 'cell', 'cgab', 'chem', 'chvf', 'chvs',
    'clas', 'clna', 'clnd', 'cnce', 'comd', 'crbs', 'diap', 'dora', 'drdd', 'dsyn',
    'edac', 'eehu', 'elii', 'emod', 'emst', 'enty', 'enzy', 'euka', 'evnt', 'famg',
    'ffas', 'fish', 'fndg', 'fngs', 'food', 'ftcn', 'genf', 'geoa', 'gngm', 'gora',
    'grpa', 'grup', 'hcpp', 'hcro', 'hlcs', 'hops', 'horm', 'humn', 'idcn', 'imft',
    'inbe', 'inch', 'inpo', 'inpr', 'irda', 'lang', 'lbpr', 'lbtr', 'mamm', 'mbrt',
    'mcha', 'medd', 'menp', 'mnob', 'mobd', 'moft', 'mosq', 'neop', 'nnon', 'npop',
    'nusq', 'ocac', 'ocdi', 'orch', 'orga', 'orgf', 'orgm', 'orgt', 'ortf', 'patf',
    'phob', 'phpr', 'phsf', 'phsu', 'plnt', 'podg', 'popg', 'prog', 'pros', 'qlco',
    'qnco', 'rcpt', 'rept', 'resa', 'resd', 'rnlw', 'sbst', 'shro', 'socb', 'sosy',
    'spco', 'tisu', 'tmco', 'topp', 'virs', 'vita', 'vtbt'
    ]
    
# Set entity types to 0 in the DataFrame
data[entity_types] = 0

In [ ]:
# Split the DataFrame into chunks to avoid resource errors
# chunk_size = 90
# chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

In [ ]:
# Extract MetaMap concepts
def extract_mm_concepts(row):
    concepts_list = []

    sents = [row['text']]

    try:
        concepts, error = mm.extract_concepts(
            sents,
            [1],
            word_sense_disambiguation=True,
            ignore_stop_phrases=True,
            composite_phrase=0,
            prune=20
        )

        concepts_list.extend(concepts)
    except Exception as e:
        print(f"Error extracting concepts: {e}")

    return concepts_list

# Update the entity type counts for each row
for index, row in data.iterrows():
    concepts = extract_mm_concepts(row)

    for concept in concepts:
        types = concept.semtypes.strip("[]").split(",")
        for type in types:
            if type in entity_types:
                data.at[index, type] += 1

# Print the updated data
print(data)

In [ ]:
# Save the updated dataframe to a new file
data.to_csv('_most_recent_data.csv', index=False)

In [ ]:
cleaned_data = data.drop(['title','text','source','index'],axis=1)

In [ ]:
cleaned_data.to_csv("metamap_data.csv", index=False)